In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split

In [2]:
path = './multilabel_modified/multilabel_classification(6)-reduced_modified.csv'
df = pd.read_csv(path)
df

,Image_Name,"Classes(motorcycle, truck, boat, bus, cycle, , , , , , , sitar, ektara, flutes, tabla, harmonium)",motorcycle,truck,boat,bus,cycle,sitar,ektara,flutes,tabla,harmonium
0,image1.jpg,bus,0,0,0,1,0,0,0,0,0,0
1,image2.jpg,sitar,0,0,0,0,0,1,0,0,0,0
2,image3.jpg,flutes,0,0,0,0,0,0,0,1,0,0
3,image4.jpg,bus,0,0,0,1,0,0,0,0,0,0
4,image5.jpg,bus,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7938,image7839.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1
7939,image7840.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1
7940,image7841.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1
7941,image7842.jpg,ektara flutes tabla harmonium,0,0,0,0,0,0,1,1,1,1


In [3]:
image_path = './multilabel_modified/images/'

In [4]:
def load_and_preprocess_image(img_name):
    img = cv2.imread(os.path.join(image_path, img_name))
    img = cv2.resize(img, (128, 128))  # Resize to 128x128
    img = img / 255.0  # Normalize the image to [0, 1]
    return img

In [5]:
images = np.array([load_and_preprocess_image(img_name) for img_name in df['Image_Name']])
labels = df.iloc[:, 2:].values

In [6]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [7]:
def create_model():
    base_model = tf.keras.applications.VGG19(include_top=False, input_shape=(128, 128, 3), weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='sigmoid')
    ])
    
    return model

model = create_model()

80134624/80134624 [==============================] - 12s 0us/step


In [8]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,224,330
Trainable params: 4,199,946
Non-trainable params: 20,024,384
_________________________________________________________________


In [9]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    verbose=1)

Epoch 1/20
199/199 [==============================] - 148s 741ms/step - loss: 0.1555 - accuracy: 0.6788 - val_loss: 0.0853 - val_accuracy: 0.8005
Epoch 2/20
199/199 [==============================] - 160s 803ms/step - loss: 0.0796 - accuracy: 0.7984 - val_loss: 0.0656 - val_accuracy: 0.8018
Epoch 3/20
199/199 [==============================] - 145s 728ms/step - loss: 0.0660 - accuracy: 0.8096 - val_loss: 0.0597 - val_accuracy: 0.8326
Epoch 4/20
199/199 [==============================] - 150s 756ms/step - loss: 0.0542 - accuracy: 0.8403 - val_loss: 0.0592 - val_accuracy: 0.8169
Epoch 5/20
199/199 [==============================] - 166s 837ms/step - loss: 0.0467 - accuracy: 0.8577 - val_loss: 0.0588 - val_accuracy: 0.8150
Epoch 6/20
199/199 [==============================] - 191s 963ms/step - loss: 0.0390 - accuracy: 0.8647 - val_loss: 0.0555 - val_accuracy: 0.8307
Epoch 7/20
199/199 [==============================] - 265s 1s/step - loss: 0.0356 - accuracy: 0.8705 - val_loss: 0.0558 - va

In [10]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

50/50 [==============================] - 27s 537ms/step - loss: 0.0670 - accuracy: 0.8320
Validation Loss: 0.06699027121067047
Validation Accuracy: 0.8319697976112366
